In [4]:
%load_ext autoreload
%autoreload 2

# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created (in the wow folder):
- **wow.log.Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **wow.log.Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **wow.query.Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **wow.query.Predicate**
  - *A Collection of functions that will be applied to those iterators*

There are also another 2 modules in addition to both modules containing classes above:
- **ui**
  - contain ui elements in this notebook

- **fights**
  - contains scripts to analyze each fight (only nerzhul for now)


In [5]:
# Initialize
import wow.ui as ui

from wow.query import Query, Predicate
from wow.log import Record, Encounter

(z, f) = ui.import_file()

# Create Data Structures

In [10]:
encounters = Encounter.parse(f.content)

0  =>  20


# Detailed Analysis

In [13]:
# Define Encounter being Analysed
encSelect = ui.pick_encounter(encounters)

In [16]:
e = encounters[encSelect.value]